# 5.1 합성곱 신경망 소개

### MNIST 숫자 이미지 분류에 convnet을 적용

**기본적인 Convnet의 형태**
- **Conv2D**와 **MaxPooling2D**층을 쌓아올린 후 마지막에 **완전연결층(FCN, Fully Connected Network)**을 쌓은 형태
- **Conv2D와 MaxPooling2D층의 출력**
    - **(height, width, channels) 크기의 3D 텐서**
    - 출력의 채널 수는 Conv2D층의 첫번째 매개변수에 의해 조절됨
- 완전연결층에 3D 출력 텐서를 주입해야 하므로 1D 텐서로 펼치는 과정이 있음

In [1]:
# Simple Convnet

from keras import layers
from keras import models

model = models.Sequential()

# Conv2D() + MaxPooling2D()
# input_shape=(image_height, image_width, image_channels)
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# 3D -> 1D
model.add(layers.Flatten())

# Dense()
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [2]:
# Data Preprocessing

from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255.

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255.

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(60000, 28, 28, 1) (60000, 10)
(10000, 28, 28, 1) (10000, 10)


In [3]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs = 5
hist = model.fit(train_images, train_labels, epochs=epochs, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 5s 91us/step - loss: 0.1622 - accuracy: 0.9506
Epoch 2/5
60000/60000 [==============================] - 4s 72us/step - loss: 0.0447 - accuracy: 0.9862
Epoch 3/5
60000/60000 [==============================] - 5s 82us/step - loss: 0.0307 - accuracy: 0.9907
Epoch 4/5
60000/60000 [==============================] - 5s 89us/step - loss: 0.0235 - accuracy: 0.9929
Epoch 5/5
60000/60000 [==============================] - 5s 84us/step - loss: 0.0178 - accuracy: 0.9944


In [4]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_loss, test_acc)

10000/10000 [==============================] - 1s 51us/step
0.02688317510868801 0.9919000267982483


**MNIST에서 Convnet이 2장의 FCN보다 좋은 테스트 정확도를 보임**